In [20]:
# imports
import scipy.io
import numpy as np               # for arrays
from numpy import linalg as LA   # for eigenvalues
import matplotlib                # for plots
import time                      # for time measurements
from PIL import Image            # for showing images
import matplotlib.pyplot as plt
from sklearn import preprocessing
import sympy
from functions import *

In [2]:
def show_img(img):
    temp = img.copy()
    temp.resize((46,56))
    im = Image.fromarray(temp.T)
    im.show()

Get training/test splits

In [3]:
# load data
mat = scipy.io.loadmat('face.mat')
raw_data = mat['X']

raw_data = np.transpose(raw_data)
N,D = raw_data.shape
C = 52 # number of classes in dataset
train_size = int(N * 0.8)
test_size = int(N * 0.2)

pca_training_data = np.empty([int(520*0.8), 2576])
pca_testing_data = np.empty([int(520*0.2), 2576])
lda_training_data = []
lda_testing_data = []

# create training and test data
for x in range(52):
    # 8/2 ratio for training and testing datasets
    lda_training_data.append(raw_data[x*10:x*10+8].copy())
    lda_testing_data.append(raw_data[x*10+8:(x+1)*10].copy())
    

lda_training_data = np.array(lda_training_data)
lda_testing_data = np.array(lda_testing_data)
pca_training_data = lda_training_data.reshape(train_size, D)
pca_testing_data = lda_testing_data.reshape(test_size,D)


We need to make sure that the generalised eigenvalue problem that we encounter when doing LDA is solvable by making sure that the within-class scatter matrix is non-singular.
We do this by first reducing the dimension of the data via low dim PCA to an M <= N - c.

In [4]:
def get_Wpca(data, out_dim):
    # low-dim PCA
    S = data.dot(data.T)
    w, v = LA.eigh(S)
    u = data.T.dot(v)
    u = u.T # the eigenvectors aren't normalised after this
    u /= LA.norm(u, ord=2, axis=0)
    
    # sort wrt abs(eigenvalue)
    id = np.argsort(np.abs(w))[::-1]
    w = w[id]
    u = u[id]
    print("shape of u lol:", u.shape)
    return u[0:out_dim].T
    

We want a projection that maximises the ratio between the between-class scatter matrix and the within class scatter matrix.
The projection W turns out to be the solutions to the generalised eigen value problem. (Found via solving the langrangian. Slide 10-11)

In [5]:
def get_Wlda(Sb_data, Sw_data, out_dim):
    #code that gets value
    lda_evals, lda_evecs = LA.eig(LA.inv(Sw_data).dot(Sb_data))
    # print("magnitud of eigenvecs: ", LA.norm(lda_evecs.T[0], ord=2, axis=0))
    lda_evecs = lda_evecs.T
#     print("lda_evals: ", lda_evals)
    
    # sort wrt abs(eigenvalue)
    id = np.argsort(np.abs(lda_evals))[::-1]
    lda_evals = lda_evals[id]
    lda_evecs = lda_evecs[id]
    print(lda_evecs.shape)
    return lda_evecs[0:out_dim].T

In [6]:
mean_all_data = pca_training_data.T.mean(axis=1).T

# between class scatter (scalar)
mean_class_data = lda_training_data.mean(axis=1)
diff_class_mean = mean_class_data - mean_all_data
print(mean_class_data.shape)
print(lda_training_data.shape)
Sb = np.dot(diff_class_mean.T, diff_class_mean)
Sb *= np.array([8])

# within class scatter (scalar)
diff_class_data = lda_training_data - mean_class_data.reshape(52,1,-1)


(52, 2576)
(52, 8, 2576)


In [7]:
Sw = np.zeros((2576, 2576));
for x in diff_class_data:
    Sw += np.dot(x.T,x)

Mpca = 40
Mlda = 416
Wpca = get_Wpca(pca_training_data, Mpca)
print(LA.norm(Wpca[0], ord=2, axis=0))
reduced_Sb = Wpca.T.dot(Sb).dot(Wpca) # symmetric matrix hopefully
reduced_Sw = Wpca.T.dot(Sw).dot(Wpca)
print("Check symmetry: ", reduced_Sw - reduced_Sw.T)
Wlda = get_Wlda(reduced_Sb, reduced_Sw, Mlda)

Wopt = Wlda.T.dot(Wpca.T).T
Wopt /= LA.norm(Wopt, ord=2, axis=1, keepdims=True)
# print("Wopt: ", Wopt.shape, " Wlda.T: ", Wlda.T.shape, " Wpca.T ", Wpca.T.shape)

shape of u lol: (416, 2576)
0.93908674141531
Check symmetry:  [[ 0.00000000e+00  4.47034836e-07  1.19209290e-07 ...  0.00000000e+00
  -9.53674316e-07 -2.38418579e-07]
 [-4.47034836e-07  0.00000000e+00  0.00000000e+00 ... -2.98023224e-08
  -5.58793545e-09  5.58793545e-09]
 [-1.19209290e-07  0.00000000e+00  0.00000000e+00 ...  2.14204192e-08
   0.00000000e+00 -1.86264515e-08]
 ...
 [ 0.00000000e+00  2.98023224e-08 -2.14204192e-08 ...  0.00000000e+00
   0.00000000e+00  3.72529030e-09]
 [ 9.53674316e-07  5.58793545e-09  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00 -3.72529030e-09]
 [ 2.38418579e-07 -5.58793545e-09  1.86264515e-08 ... -3.72529030e-09
   3.72529030e-09  0.00000000e+00]]
(40, 40)


In [8]:
print(Wopt)

[[-0.17914507  0.06420822  0.06010227 ...  0.13120022 -0.23442844
   0.15990052]
 [-0.30870077 -0.01304439 -0.07163005 ...  0.02233892 -0.07113221
   0.1399904 ]
 [-0.28683954  0.04344889 -0.17833653 ...  0.08020104  0.00711227
   0.08262923]
 ...
 [ 0.14111634 -0.06243067 -0.1739653  ...  0.01526845 -0.16520695
   0.28920911]
 [ 0.11727967 -0.06132396  0.10691144 ... -0.01479688 -0.2126098
   0.19340099]
 [ 0.17535864 -0.01216471  0.12793412 ... -0.18851593 -0.09175644
   0.20543917]]


We can now express each data point as product of a weight vector with Wopt. This weight vector can be used to classify each data point using nearest neighbour

In [9]:
training_face_weights = Wopt.T.dot(pca_training_data.T)
testing_face_weights = Wopt.T.dot(pca_testing_data.T)

print(training_face_weights.shape)
print(testing_face_weights.shape)

(40, 416)
(40, 104)


result = class_rate(training_face_weights.real, testing_face_weights.T.real)
print("classification rate: ", 100*np.sum(result)/len(result), "%.")

In [21]:
results = vary_Mlda(training_face_weights.real, testing_face_weights.real)
plot_data("yo","hnth","onetu",results)

ValueError: query data dimension must match training data dimension

In [ ]:
print(results)